In [9]:
import pdfplumber
import re
import pandas as pd
print(pdfplumber.__version__)

0.11.4


## 载入pdf

In [10]:
pdf = pdfplumber.open("RfQ_0407-2024d_204423_vessel_HELGOLAND_2ZJV(1).pdf")

In [11]:

pageNum=len(pdf.pages)

In [12]:
totalStr=''
core_pat = re.compile(r"Qty Price(.*)", re.DOTALL)
for i in range(0,pageNum):
    print(f'''处理第 {i} 页''')
    p0 = pdf.pages[i]
    text = p0.extract_text(keep_blank_chars=True)
    # if i==2:
    #     print(text)
    
    core = re.search(core_pat, text)
    if core:
        currentStr=core.group(1)
        currentStr=re.sub(r'Page (\d+)/(\d+)', '', currentStr)
        currentStr=re.sub(r'Total: \d+ item\(s\)\.', '', currentStr)
        totalStr += currentStr  # 获取第一个捕获组的内容
        #print("Matched content:", core)
    else:
        print("No match found.")

处理第 0 页
No match found.
处理第 1 页
处理第 2 页
处理第 3 页


In [13]:

#print(totalStr)
lines = re.split(r'\r\n|\n', totalStr)
# 去除空字符串
filtered_lines = [line for line in lines if line]

# 输出结果
print(filtered_lines)

['1 100,00pcs 65.160.03 BOLTS STAINLESS STEEL M6x50.', '2 100,00pcs 65.160.06 BOLTS STAINLESS STEEL M8x40.', "3 200,00pcs 65.160.07 'BOLTS STAINLESS STEEL M8x50.", "4 200,00pcs 65.160.08 'BOLTS STAINLESS STEEL M8x75.", '5 200,00pcs N/A BOLTS STAINLESS STEEL M5x20.', "6 50,00pcs 65.160.36 'BOLTS STAINLESS STEEL M16X60.", "7 50,00pcs N/A 'BOLTS STAINLESS STEEL M20X50.", "8 50,00pcs N/A 'BOLTS STAINLESS STEEL M20X80.", "9 100,00pcs 65.171.03 'NUTS STAINLESS STEEL M6.", "10 300,00pcs 65.171.04 'NUTS STAINLESS STEEL M8.", "11 200,00pcs 65.171.05 'NUTS STAINLESS STEEL M10.", "12 50,00pcs N/A 'NUTS STAINLESS STEEL M20.", "13 500,00pcs 65.181.01 'WASHERS STAINLESS STEEL M4.", "14 500,00pcs 65.181.02 'WASHERS STAINLESS STEEL M5.", "15 200,00pcs 65.181.05 'WASHERS STAINLESS STEEL M10.", "16 200,00pcs 65.181.06 'WASHERS STAINLESS STEEL M12.", "17 200,00pcs N/A 'WASHERS STAINLESS STEEL M20.", "18 1000,00pcs 65.173.04 'M8 NUTS - HEX HEAD LOCKING (NYLON)", 'STAINLESS STEEL.', '19 5,00PCS 351222 COUP

In [14]:
res_lines=[]
num_list=0
for item in filtered_lines:
    # 正则表达式模式
    pattern = r'(^\d+\s)'
    pat = re.compile(pattern, re.DOTALL)
    findStr = re.search(pat, item)
    if findStr:
        r=findStr.group(0)
        if int(r)>0:
            #print(int(r))
            #print("num_list:",num_list)
            #比较当前的数字是否是序号
            if num_list+1==int(r):
                num_list=int(r)
                #print('new:',r)
                res_lines.append(item)
            else:
                if(len(res_lines)>0):
                    res_lines[len(res_lines)-1]+='\n'+item
                
        else:
             if(len(res_lines)>0):
                res_lines[len(res_lines)-1]+='\n'+item
             #print("不是数字.")
        
    else:
        if(len(res_lines)>0):
            res_lines[len(res_lines)-1]+='\n'+item
        #print("No match found.")

print(res_lines)


['1 100,00pcs 65.160.03 BOLTS STAINLESS STEEL M6x50.', '2 100,00pcs 65.160.06 BOLTS STAINLESS STEEL M8x40.', "3 200,00pcs 65.160.07 'BOLTS STAINLESS STEEL M8x50.", "4 200,00pcs 65.160.08 'BOLTS STAINLESS STEEL M8x75.", '5 200,00pcs N/A BOLTS STAINLESS STEEL M5x20.', "6 50,00pcs 65.160.36 'BOLTS STAINLESS STEEL M16X60.", "7 50,00pcs N/A 'BOLTS STAINLESS STEEL M20X50.", "8 50,00pcs N/A 'BOLTS STAINLESS STEEL M20X80.", "9 100,00pcs 65.171.03 'NUTS STAINLESS STEEL M6.", "10 300,00pcs 65.171.04 'NUTS STAINLESS STEEL M8.", "11 200,00pcs 65.171.05 'NUTS STAINLESS STEEL M10.", "12 50,00pcs N/A 'NUTS STAINLESS STEEL M20.", "13 500,00pcs 65.181.01 'WASHERS STAINLESS STEEL M4.", "14 500,00pcs 65.181.02 'WASHERS STAINLESS STEEL M5.", "15 200,00pcs 65.181.05 'WASHERS STAINLESS STEEL M10.", "16 200,00pcs 65.181.06 'WASHERS STAINLESS STEEL M12.", "17 200,00pcs N/A 'WASHERS STAINLESS STEEL M20.", "18 1000,00pcs 65.173.04 'M8 NUTS - HEX HEAD LOCKING (NYLON)\nSTAINLESS STEEL.", '19 5,00PCS 351222 COUPLE

## 处理各个序号下的数据

In [15]:
data=[]
class RowData:
    def __setattr__(self, name, value):
        self.__dict__[name] = value
for item in res_lines:
    rowData = RowData()
    #根据空格分解
     # 正则表达式模式
    lines = re.split(r' ', item)
    if len(lines)>0:
        pattern = r"(\d+(?:,\d+)*(?:\.\d+)?)\s*([\w\s]+[\w\s]*[^\w\s]?)"
        #序号
        rowData.Pos = int(lines[0])
        
        match = re.match(pattern, lines[1])
        if match:
            number = match.group(1)
            unit = match.group(2)
            #数量
            rowData.Quantity=number
            #单位
            rowData.Unit=unit
        #编码
        rowData.Part_number=lines[2]
        #描述
        rowData.Description=' '.join(lines[2:])
    data.append(rowData)

print(data)

[<__main__.RowData object at 0x0000023FD5A20990>, <__main__.RowData object at 0x0000023FD5E37210>, <__main__.RowData object at 0x0000023FD5E36510>, <__main__.RowData object at 0x0000023FD5E37550>, <__main__.RowData object at 0x0000023FD5E37250>, <__main__.RowData object at 0x0000023FD5E313D0>, <__main__.RowData object at 0x0000023FD5E30610>, <__main__.RowData object at 0x0000023FD5E30110>, <__main__.RowData object at 0x0000023FD5E30090>, <__main__.RowData object at 0x0000023FD5E312D0>, <__main__.RowData object at 0x0000023FD5E30390>, <__main__.RowData object at 0x0000023FD5E30290>, <__main__.RowData object at 0x0000023FD47C4C90>, <__main__.RowData object at 0x0000023FD47C5BD0>, <__main__.RowData object at 0x0000023FD6C02150>, <__main__.RowData object at 0x0000023FD6C02550>, <__main__.RowData object at 0x0000023FD6C01C10>, <__main__.RowData object at 0x0000023FD6C02B50>, <__main__.RowData object at 0x0000023FD6C03950>, <__main__.RowData object at 0x0000023FD6C0C990>, <__main__.RowData o

## 导出excel

In [16]:
# 将对象数组转换为 DataFrame
df = pd.DataFrame([
    (row.Pos, row.Quantity, row.Unit,row.Part_number, row.Description)
    for row in data
], columns=['Pos', 'Quantity','Unit', 'Part_number', 'Description'])

# 导出到 Excel 文件
df.to_excel('output.xlsx', index=False)

# 输出结果
print("数据已导出到 output.xlsx")

数据已导出到 output.xlsx
